# Import 套件 & Load Data

In [ ]:
import pickle

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
!git clone https://github.com/edlin0317/taichung-house-price.git

fatal: destination path 'taichung-house-price' already exists and is not an empty directory.


# 產出模型

In [ ]:
!wget -O data.csv https://raw.githubusercontent.com/edlin0317/taichung-house-price/main/B-B28-臺中市大里區.csv

--2021-06-16 17:00:28--  https://raw.githubusercontent.com/edlin0317/taichung-house-price/main/B-B28-%E8%87%BA%E4%B8%AD%E5%B8%82%E5%A4%A7%E9%87%8C%E5%8D%80.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5480945 (5.2M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   5.23M  20.4MB/s    in 0.3s    

2021-06-16 17:00:28 (20.4 MB/s) - ‘data.csv’ saved [5480945/5480945]



In [ ]:
def make_model(filename, year):
    df = pd.read_csv(filename)

    filename = filename.split('/')[-1]
    print('----------------------------')
    print(f'{filename} | year {year}')
    # filename = 'data.csv'
    # year = 101
    

    def y_convert(t):
        t = str(t)
        year = int(t[:-2])
        return year

    y_convert('11004')

    def percentage_to_float(percent):
        if type(percent) == str:
            return int(percent.replace('.', '').replace('%',''))/10000
        return percent

    percentage_to_float('18.88%')

    def price_to_10k(price):
        return round(price/10000)

    price_to_10k(157200)

    def total_floor(f): #tf
        return f.split('/')[1]

    def digit(c):
        pair = {
            '一':1, '二':2, '三':3, '四':4, '五':5, '六':6, '七':7, '八':7, '九':9, '十':10 
        }
        return pair[c]

    def floor_to_int(f):
        f = f.replace('層', '')
        sum = 0
        if len(f) == 3:             #ex: 二十七
            sum+=digit(f[0])*10     #f[0] = 二
            sum+=digit(f[2])        #f[2] = 七
        if len(f) == 2:
            if not f[0] == '十':    #ex 二十
                sum+=digit(f[0])*10 #f[0] = 二
            else:                   #ex 十七
                sum+=10
                sum+=digit(f[1])
        if len(f) == 1:             #ex 七
            sum+=digit(f)           #f[0] = 七
        return sum


    #bs - 將字串轉float
    df['bs'] = df['bs'].apply(percentage_to_float)
    #將年月轉換成年
    df['year'] = df['e'].apply(y_convert)
    #將總價中的逗號移除
    df['tp'] = df['tp'].str.replace(',','').astype(int)
    #將每坪單價中的逗號移除
    df['p'] = df['p'].str.replace(',','').astype(int)#.apply(price_to_10k)
    #坪數 s 改為float
    df['s'] = df['s'].str.replace(',','').astype(float)
    #丟掉不相關欄位
    df = df.drop(columns=['sq', 'AA12', 'cp', 'es', 'city', 'district_code', 'pimg', 'type', 'id'])
    # df.rename(columns={
    #     'a':'地址門牌(a)',
    #     'AA11':'用途(AA11)',
    #     'b':'建物類型(b)',
    #     'bs':'主建物占總面積比例(bs)',
    #     'e':'交易年月(e)',
    #     'el':'有無電梯(el)',
    #     'f':'交易樓層/總樓層(f)',
    #     'fi':'unknown(fi)',
    #     'g':'屋齡(g)',
    #     'j':'土地筆數(j)',
    #     'k':'建物筆數(k)',
    #     'l':'車位筆數(l)',
    #     'lat':'緯度(lat)',
    #     'lon':'經度(lon)',
    #     'm':'管委會(m)',
    #     'p':'每坪單價(p)',
    #     'r':'unknown(r)',
    #     's':'總坪數(s)',
    #     't':'類型(t)',
    #     'tp':'交易總價(tp)',
    #     'v':'格局(v)'
    #     }).head(10)

    # 只保留「住宅大樓」 其他類型樣本數不足 - 剔除
    #df = df.drop(df.loc[(df['b'] == '住宅大樓(11層含以上有電梯)')].index)
    df = df.drop(df.loc[(df['b'] == '透天厝')].index)
    df = df.drop(df.loc[(df['b'] == '華廈(10層含以下有電梯)')].index)
    df = df.drop(df.loc[(df['b'] == '公寓(5樓含以下無電梯)')].index)
    df = df.drop(df.loc[(df['b'] == '套房(1房(1廳)1衛)')].index)
    df = df.drop(df.loc[(df['b'] == '店面（店舖)')].index)
    df = df.drop(df.loc[(df['b'] == '辦公商業大樓')].index)
    df = df.drop(df.loc[(df['b'] == '其他')].index)
    df = df.drop(df.loc[(df['b'] == '廠辦')].index)
    df = df.drop(df.loc[(df['b'] == '工廠')].index)
    df = df.drop(df.loc[(df['b'] == '倉庫')].index)
    
    print(f'住宅大樓樣本數 : {len(df)}')
    if len(df) < 5000:
        print('樣本數不足!')
        return False

    #只保留非NaN
    df = df[df['AA11'].notna()]
    #只保留「住」
    df = df.drop(df.loc[(df['AA11'] == '其他')].index)
    df = df.drop(df.loc[(df['AA11'] == '商')].index)
    df = df.drop(df.loc[(df['AA11'] == '工')].index)
    df = df.drop(df.loc[(df['AA11'] == '住宅區')].index)
    df = df.drop(df.loc[(df['AA11'] == '農')].index)
    df = df.drop(df.loc[(df['AA11'] == '山坡地保育區')].index)
    df = df.drop(df.loc[(df['AA11'] == '特定農業區')].index)

    #只取某一年
    df_101 = df.loc[(df['year'] == year)]
    df_101

    #清除格局空值
    df_101 = df_101[df_101['v'].notna()]
    df_101

    def room(v):
        return int(v.split('房')[0])

    def living(v):
        return int(v.split('房')[1].split('廳')[0])

    def toilet(v):
        return int(v.split('廳')[1].split('衛')[0])

    #房數 room
    df_101['room'] = df_101['v'].apply(room)
    #廳數 living
    df_101['living'] = df_101['v'].apply(living)
    #衛數 toilet
    df_101['toilet'] = df_101['v'].apply(toilet)

    #定義極值
    s_lower = df_101.s.quantile(0.05)
    s_median = df_101.s.quantile(0.5)
    s_upper = df_101.s.quantile(0.95)
    # print(s_lower, s_median, s_upper)

    p_lower = df_101.p.quantile(0.05)
    p_median = df_101.p.quantile(0.5)
    p_upper = df_101.p.quantile(0.95)
    # print(p_lower, p_median, p_upper)

    tp_lower = df_101['tp'].quantile(0.05)
    tp_median = df_101['tp'].quantile(0.5)
    tp_upper = df_101['tp'].quantile(0.95)
    # print(tp_lower, tp_median, tp_upper)

    #剔除極值 - 捨去坪數 s 太大/太小
    df_101 = df_101.drop(df_101.loc[(df_101['s'] < s_lower) & True].index)
    df_101 = df_101.drop(df_101.loc[df_101['s'] > s_upper].index)
    # print(len(df_101))
    #剔除極值 - 捨去每坪單價 p 太大/太小
    df_101 = df_101.drop(df_101.loc[df_101['p'] < p_lower].index)
    df_101 = df_101.drop(df_101.loc[df_101['p'] > p_upper].index)
    # print(len(df_101))
    #剔除極值 - 捨去總價 tp 太大/太小
    df_101 = df_101.drop(df_101.loc[df_101['tp'] < tp_lower].index)
    df_101 = df_101.drop(df_101.loc[df_101['tp'] > tp_upper].index)
    # print(len(df_101))

    #補缺值
    df_101['bs'] = df_101['bs'].fillna(1) #假設主建物比=1
    df_101['g'] = df_101['g'].fillna(0) #假設屋齡=0

    #計算實坪 (主建物坪)
    df_101['real_s'] = df_101['bs'] * df_101['s']

    #取訓練用欄位
    tmp = df_101[['bs','g','l','lat','lon','s','room','living','toilet', 'real_s','tp','p']]

    #建立模型儲存位置
    import os
    os.makedirs('model', exist_ok=True)

    #訓練模型
    from sklearn import linear_model
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.tree import DecisionTreeRegressor
    import xgboost as xgb
    from sklearn.model_selection import train_test_split

    #預測總價
    X = tmp.drop(columns=['tp','p']).values
    y = tmp['tp'].values

    #找出最佳模型
    best_score = 0
    best_model = None

    for i in range(10):
        X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3)
        #model = linear_model.LinearRegression()
        #model = KNeighborsRegressor(n_neighbors=3)
        #model = DecisionTreeRegressor()
        model = xgb.XGBRegressor(verbosity = 0) #suppress warning
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        score = round(r2_score(Y_test, Y_pred),2)
        #print(score)
        if score > best_score:
            best_model = model
            best_score = score

    #模型表現
    print(f'test samples size : {len(X_test)}')
    print(f'best score : {best_score}')

    #儲存模型
    model_name = f"{filename.split('.')[0]}_tp_{year}.pickle"
    save_path = 'model/'
    pickle.dump(model, open(save_path + model_name, 'wb'))
    print(f'model saved at: {save_path + model_name}')
    print('=================================')

    #預測單價
    X = tmp.drop(columns=['tp','p']).values
    y = tmp['p'].values

     #找出最佳模型
    best_score = 0
    best_model = None

    for i in range(10):
        X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3)
        #model = linear_model.LinearRegression()
        #model = KNeighborsRegressor(n_neighbors=3)
        #model = DecisionTreeRegressor()
        model = xgb.XGBRegressor(verbosity = 0) #suppress warning
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        score = round(r2_score(Y_test, Y_pred),2)
        #print(score)
        if score > best_score:
            best_model = model
            best_score = score

    #模型表現
    print(f'test samples size : {len(X_test)}')
    print(f'best score : {best_score}')

    #儲存模型
    model_name = f"{filename.split('.')[0]}_p_{year}.pickle"
    save_path = 'model/'
    pickle.dump(model, open(save_path + model_name, 'wb'))
    print(f'model saved at: {save_path + model_name}')
    print('=================================')

make_model('data.csv', 101)

----------------------------
data.csv | year 101
住宅大樓樣本數 : 6815
test samples size : 33
best score : 0.93
model saved at: model/data_tp_101.pickle
test samples size : 33
best score : 0.79
model saved at: model/data_p_101.pickle


In [ ]:
# import os
# path = 'taichung-house-price'
# files = os.listdir(path)

# for filename in files:
#     for year in range(101, 111):
#         if '.csv' in filename:
#             try:
#                 make_model(f'{path}/{filename}', year)
#             except Exception as e:
#                 print(str(e))

In [ ]:
# !zip -r model.zip model

In [ ]:
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def price_predict(lat, lon, model, bs=0.5, g=0, l=0, s=30, room=3, living=2, toilet=2, real_s=None):
    # load the model from disk
    
    #calculate bs from s and real_s
    if real_s != None:
        bs = real_s / s

    #auto calculate real_s
    if real_s == None:
        real_s = bs * s

    data = {
        'bs':bs,#主建物比
        'g':g,#屋齡
        'l':l,#車位數
        'lat':lat,#緯度
        'lon':lon,#經度
        's':s,#總坪數
        'room':room,#房
        'living':living,#廳
        'toilet':toilet,#衛
        'real_s':real_s#主建物坪數
    }
    data = list(data.values())
    result = model.predict(data)[0]
    return result



In [ ]:
model_filename = 'B-B03-臺中市南區_p_101.pickle'
model = pickle.load(open(model_filename, 'rb'))
price_predict(model=model, lat=24.118528, lon=120.658202, s=39.17)

127478.1

In [ ]:
price_predict(model=model, lat=24.552528, lon=120.628202, s=31)

157940.02

# model api on flask

In [1]:
!rm -r taichung-house-price

rm: cannot remove 'taichung-house-price': No such file or directory


In [2]:
!git clone https://github.com/edlin0317/taichung-house-price.git

Cloning into 'taichung-house-price'...
remote: Enumerating objects: 205, done.
remote: Total 205 (delta 0), reused 0 (delta 0), pack-reused 205
Receiving objects: 100% (205/205), 23.36 MiB | 17.68 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [6]:
!pip install pyngrok

     |████████████████████████████████| 747kB 5.1MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19262 sha256=ebfc47a9a364b06f45afc1ec26bf734fcdd238e5e4242b876c82b165e85aeb09
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [7]:
from pyngrok import ngrok
ngrok.kill()
#ngrok.set_auth_token("api key")
ngrok.connect(5000)
ngrok.get_tunnels()

[<NgrokTunnel: "https://2f24df1fcdec.ngrok.io" -> "http://localhost:5000">,
 <NgrokTunnel: "http://2f24df1fcdec.ngrok.io" -> "http://localhost:5000">]

In [52]:
ngrok.get_tunnels()[0].public_url

'https://7b688884fa0e.ngrok.io'

In [53]:
import os
import pickle

def model_file(city, district, year, type_):
    path = 'taichung-house-price/model'
    files = os.listdir(path)
    for file in files:
        if (city in file) and (district in file) and (year in file) and (f'_{type_}_' in file):
            model_filename = f'{path}/{file}'
            model = pickle.load(open(model_filename, 'rb'))
            return model
    return None


model_file(city='B', district='B07', year='108', type_='p')



[05:13:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=0)

In [ ]:
print(ngrok.get_tunnels()[0].public_url + '/model_api?city=B&district=B07&year=108&lat=24.124566887961176&lon=120.67507110914983&bs=0.6&g=0&l=1&room=2&living=1&toilet=1&s=50&real_s=30')

from flask import Flask, jsonify, request
import requests
import json

app = Flask(__name__)
#json 中文回傳
app.config['JSON_AS_ASCII'] = False

@app.route('/')
def home():
    return 'Welcome to Taichung House Price Prediction Model API !'

def price_predict(model, lat, lon, bs=0.5, g=0, l=0, room=3, living=2, toilet=2, s=50, real_s=None):
    #calculate bs from s and real_s
    if real_s != None:
        bs = real_s / s

    #auto calculate real_s
    if real_s == None:
        real_s = bs * s

    data = {
        'bs':bs,#主建物比
        'g':g,#屋齡
        'l':l,#車位數
        'lat':lat,#緯度
        'lon':lon,#經度
        's':s,#總坪數
        'room':room,#房
        'living':living,#廳
        'toilet':toilet,#衛
        'real_s':real_s#主建物坪數
    }
    data = list(data.values())
    result = model.predict(data)[0]
    return result
    
@app.route('/model_api', methods=['GET', 'POST'])
def model_api():
    try:
        #read model
        city = request.args.get('city')
        district = request.args.get('district')
        year = str(request.args.get('year'))
        model_p = model_file(city=city, district=district, year=year, type_='p')
        model_tp = model_file(city=city, district=district, year=year, type_='tp')
        #specs
        lat = float(request.args.get('lat'))
        lon = float(request.args.get('lon'))
        bs = float(request.args.get('bs', default = 0.5))
        g = int(request.args.get('g', default = 0))
        l = int(request.args.get('l', default = 0))
        room = int(request.args.get('room', default = 3))
        living = int(request.args.get('living', default = 2))
        toilet = int(request.args.get('toilet', default = 2))
        s = float(request.args.get('s', default = 50))
        real_s = float(request.args.get('real_s', default = bs * s))
        #predict p and tp
        p = price_predict(model_p, lat=lat, lon=lon, bs=bs, g=g, l=l, room=room, living=living, toilet=toilet, s=s, real_s=real_s)
        tp = price_predict(model_tp, lat=lat, lon=lon, bs=bs, g=g, l=l, room=room, living=living, toilet=toilet, s=s, real_s=real_s)
        #result
        data = {
            'p':int(p),
            'tp':int(tp)
        }
        return jsonify(data)
    except Exception as e:
        return str(e)

if __name__ == '__main__':
    app.run()

https://7b688884fa0e.ngrok.io/model_api?city=B&district=B07&year=108&lat=24.124566887961176&lon=120.67507110914983&bs=0.6&g=0&l=1&room=2&living=1&toilet=1&s=50&real_s=30
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jun/2021 05:13:47] "GET /model_api?city=B&district=B07&year=108&lat=24.124566887961176&lon=120.67507110914983&bs=0.6&g=0&l=1&room=2&living=1&toilet=1&s=50&real_s=30 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2021 05:13:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Jun/2021 05:15:06] "GET /model_api?city=B&district=B07&year=108&lat=24.124566887961176&lon=120.67507110914983&bs=0.6&g=0&l=1&room=2&living=1&toilet=1&s=50&real_s=30 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2021 05:25:27] "POST /model_api HTTP/1.1" 405 -
127.0.0.1 - - [17/Jun/2021 05:25:34] "POST /model_api HTTP/1.1" 405 -
127.0.0.1 - - [17/Jun/2021 05:29:10] "GET /model_api?fname=adsa&lname=asdsa HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2021 05:32:40] "GET /model_api HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2021 05:33:25] "GET /model_api?city=B&district=B07&year=108&lat=24.124566887961176&lon=120.67507110914983&bs=0.6&g=0&l=1&room=2&living=1&toilet=1&s=

In [ ]:
import os
path = 'taichung-house-price/model'
files = os.listdir(path)
files

In [3]:
!pip freeze > requirements.txt